# Fraud Feature Engineering

This notebook demonstrates four examples from the book-style snippets:

A) IP vs Billing Country Mismatch (SQL + pandas)

B) Count distinct cards seen per IP (SQL + pandas)

C) Transactions per rolling window (sliding window) — using pandas to mimic SQL window RANGE

D) New device & first_seen logic (SQL + pandas)

Each section contains the **Postgres SQL** you would run in production (as a reference) followed by an executable pandas implementation on mock data so you can run it locally.

In [ ]:

# Create mock tables: transactions, users, ip_geo, devices
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

np.random.seed(42)

# Helper to create timestamps
base = datetime(2025, 9, 24, 10, 0, 0)

# Create transactions for 5 users with various times, IPs, cards, devices
rows = []
users = ['u1','u2','u3','u4','u5']
ips = ['1.2.3.4','1.2.3.5','203.0.113.5','198.51.100.12','10.0.0.5']
cards = ['card_a','card_b','card_c','card_d','card_e','card_f']
devices = ['dev1','dev2','dev3','dev4','dev5','dev6']

for uid in users:
    # create between 6 to 12 txns per user, spread across 0-30 minutes
    n = np.random.randint(6,13)
    for i in range(n):
        tx_time = base + timedelta(minutes=int(np.random.exponential(scale=6)))
        rows.append({
            'transaction_id': f"tx_{uid}_{i}",
            'user_id': uid,
            'tx_time': tx_time,
            'amount': float(np.random.choice([10,20,30,50,100,200,500], p=[0.2,0.2,0.15,0.2,0.15,0.05,0.05])),
            'ip': np.random.choice(ips, p=[0.25,0.1,0.3,0.2,0.15]),
            'card_hash': np.random.choice(cards),
            'device_id': np.random.choice(devices)
        })

transactions = pd.DataFrame(rows)
# Ensure tx_time sorted per user
transactions = transactions.sort_values(['user_id','tx_time']).reset_index(drop=True)

# Users table with billing_country
users_df = pd.DataFrame({
    'user_id': users,
    'billing_country': ['MY','MY','SG','MY','ID']
})

# ip_geo mapping
ip_geo = pd.DataFrame({
    'ip': ips,
    'country': ['SG','SG','MY','US','MY'],
    'asn': ['AS100','AS100','AS200','AS300','AS200']
})

# devices table (simulate some devices first seen earlier)
device_first = transactions.groupby('device_id')['tx_time'].min().reset_index().rename(columns={'tx_time':'first_seen'})
device_first['first_seen'] = device_first['first_seen'] - pd.to_timedelta(np.random.randint(0,48,size=len(device_first)), unit='h')

# show the created dataframes
transactions.head(12)


,transaction_id,user_id,tx_time,amount,ip,card_hash,device_id
0,tx_u1_2,u1,2025-09-24 10:00:00,500.0,198.51.100.12,card_f,dev2
1,tx_u1_1,u1,2025-09-24 10:01:00,10.0,10.0.0.5,card_d,dev3
2,tx_u1_3,u1,2025-09-24 10:01:00,10.0,1.2.3.5,card_f,dev5
3,tx_u1_9,u1,2025-09-24 10:01:00,50.0,203.0.113.5,card_c,dev6
4,tx_u1_11,u1,2025-09-24 10:01:00,50.0,1.2.3.4,card_b,dev2
5,tx_u1_6,u1,2025-09-24 10:02:00,500.0,203.0.113.5,card_e,dev3
6,tx_u1_10,u1,2025-09-24 10:02:00,10.0,198.51.100.12,card_f,dev6
7,tx_u1_4,u1,2025-09-24 10:03:00,20.0,203.0.113.5,card_b,dev4
8,tx_u1_8,u1,2025-09-24 10:04:00,20.0,1.2.3.4,card_b,dev4
9,tx_u1_7,u1,2025-09-24 10:06:00,30.0,1.2.3.4,card_a,dev4


In [ ]:
device_first.head(15)

,device_id,first_seen
0,dev1,2025-09-24 00:00:00
1,dev2,2025-09-23 07:00:00
2,dev3,2025-09-23 10:01:00
3,dev4,2025-09-23 12:01:00
4,dev5,2025-09-23 04:00:00
5,dev6,2025-09-23 05:01:00


## A — IP vs Billing Country Mismatch

### Postgres SQL (production reference)

```sql
-- Feature: ip_billing_mismatch (boolean)
SELECT
  t.transaction_id,
  t.user_id,
  t.tx_time,
  t.ip,
  ipg.country AS ip_country,
  u.billing_country,
  (ipg.country IS DISTINCT FROM u.billing_country) AS ip_billing_mismatch
FROM transactions t
LEFT JOIN ip_geo ipg ON t.ip = ipg.ip
LEFT JOIN users u ON t.user_id = u.user_id
    
WHERE t.tx_time BETWEEN CURRENT_DATE - INTERVAL '30 days' AND CURRENT_DATE;
```

### pandas implementation (executable):

In [ ]:

# Compute ip_billing_mismatch in pandas
merged = transactions.merge(ip_geo, on='ip', how='left').merge(users_df, on='user_id', how='left')
merged['ip_billing_mismatch'] = merged['country'] != merged['billing_country']
# Treat NaN (unknown ip geo) as False for mismatch for this example; in production you may treat nulls specially
merged['ip_billing_mismatch'] = merged['ip_billing_mismatch'].fillna(False)
merged[['transaction_id','user_id','tx_time','ip','country','billing_country','ip_billing_mismatch']].head(12)


,transaction_id,user_id,tx_time,ip,country,billing_country,ip_billing_mismatch
0,tx_u1_2,u1,2025-09-24 10:00:00,198.51.100.12,US,MY,True
1,tx_u1_1,u1,2025-09-24 10:01:00,10.0.0.5,MY,MY,False
2,tx_u1_3,u1,2025-09-24 10:01:00,1.2.3.5,SG,MY,True
3,tx_u1_9,u1,2025-09-24 10:01:00,203.0.113.5,MY,MY,False
4,tx_u1_11,u1,2025-09-24 10:01:00,1.2.3.4,SG,MY,True
5,tx_u1_6,u1,2025-09-24 10:02:00,203.0.113.5,MY,MY,False
6,tx_u1_10,u1,2025-09-24 10:02:00,198.51.100.12,US,MY,True
7,tx_u1_4,u1,2025-09-24 10:03:00,203.0.113.5,MY,MY,False
8,tx_u1_8,u1,2025-09-24 10:04:00,1.2.3.4,SG,MY,True
9,tx_u1_7,u1,2025-09-24 10:06:00,1.2.3.4,SG,MY,True


## B — Count distinct cards seen per IP

### Postgres SQL (production reference)

```sql
SELECT
  t.ip,
  COUNT(DISTINCT t.card_hash) AS distinct_cards_seen,
  COUNT(*) AS tx_count_30d
FROM transactions t
WHERE t.tx_time >= NOW() - INTERVAL '30 days'
GROUP BY t.ip
HAVING COUNT(DISTINCT t.card_hash) > 2
ORDER BY distinct_cards_seen DESC;
```

### pandas implementation (executable):

In [ ]:

# Count distinct cards per IP (last 30 days simulated here with full df)
ip_agg = transactions.groupby('ip').agg(
    distinct_cards_seen = ('card_hash','nunique'),
    tx_count_30d = ('transaction_id','count')
).reset_index().sort_values('distinct_cards_seen', ascending=False)
ip_agg


,ip,distinct_cards_seen,tx_count_30d
0,1.2.3.4,5,11
4,203.0.113.5,5,15
2,10.0.0.5,4,4
3,198.51.100.12,4,7
1,1.2.3.5,3,3


## C — Transactions per rolling window (sliding window)

### Postgres SQL (production reference)

```sql
SELECT
  t.*,
  COUNT(*) OVER (
    PARTITION BY user_id
    ORDER BY tx_time
    RANGE BETWEEN INTERVAL '10 minutes' PRECEDING AND CURRENT ROW
  ) AS txn_count_last_10min
FROM transactions t
WHERE tx_time >= now() - interval '14 days';
```

### pandas implementation (executable):
We compute, per user, the count of transactions in the 10-minute window ending at each transaction.

In [ ]:

# Compute rolling 10-minute count per user using pandas.
df = transactions.copy()
df['tx_time'] = pd.to_datetime(df['tx_time'])

def rolling_count_10min(group):
    # set time index
    g = group.set_index('tx_time').sort_index()
    # rolling count over 10 minute window
    # use '10min' window on the index and count rows
    g['txn_count_last_10min'] = g['amount'].rolling('10min').count().astype(int)
    return g.reset_index()

df_roll = df.groupby('user_id', group_keys=False).apply(rolling_count_10min).reset_index(drop=True)
df_roll[['transaction_id','user_id','tx_time','amount','txn_count_last_10min']].head(30)


/tmp/ipython-input-3580143588.py:13: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_roll = df.groupby('user_id', group_keys=False).apply(rolling_count_10min).reset_index(drop=True)


,transaction_id,user_id,tx_time,amount,txn_count_last_10min
0,tx_u1_2,u1,2025-09-24 10:00:00,500.0,1
1,tx_u1_1,u1,2025-09-24 10:01:00,10.0,2
2,tx_u1_3,u1,2025-09-24 10:01:00,10.0,3
3,tx_u1_9,u1,2025-09-24 10:01:00,50.0,4
4,tx_u1_11,u1,2025-09-24 10:01:00,50.0,5
5,tx_u1_6,u1,2025-09-24 10:02:00,500.0,6
6,tx_u1_10,u1,2025-09-24 10:02:00,10.0,7
7,tx_u1_4,u1,2025-09-24 10:03:00,20.0,8
8,tx_u1_8,u1,2025-09-24 10:04:00,20.0,9
9,tx_u1_7,u1,2025-09-24 10:06:00,30.0,10


## D — New device & first_seen logic

### Postgres SQL (production reference)

```sql
WITH device_first AS (
  SELECT device_id, MIN(tx_time) AS first_seen
  FROM transactions
  GROUP BY device_id
    )
SELECT t.transaction_id, t.user_id, t.device_id, t.amount, d.first_seen,
       (t.tx_time - d.first_seen) < INTERVAL '24 hours' AS device_new_last_24h
FROM transactions t
LEFT JOIN device_first d USING (device_id)
WHERE t.tx_time >= NOW() - INTERVAL '7 days'
  AND (t.tx_time - d.first_seen) < INTERVAL '24 hours'
  AND t.amount > 300;
```

### pandas implementation (executable):

In [ ]:

# Compute device first_seen and flag transactions where device is new in last 24h and amount > threshold
df = transactions.copy()
df['tx_time'] = pd.to_datetime(df['tx_time'])

device_first = df.groupby('device_id').agg(first_seen=('tx_time','min')).reset_index()

df_dev = df.merge(device_first, on='device_id', how='left')
df_dev['device_age_hours'] = (df_dev['tx_time'] - df_dev['first_seen']).dt.total_seconds() / 3600.0
df_dev['device_new_last_24h'] = df_dev['device_age_hours'] < 24
# show transactions that are new-device and amount > 300
df_dev_filtered = df_dev[(df_dev['device_new_last_24h']) & (df_dev['amount'] > 300)]
df_dev_filtered[['transaction_id','user_id','device_id','tx_time','amount','first_seen','device_age_hours','device_new_last_24h']].head(20)


,transaction_id,user_id,device_id,tx_time,amount,first_seen,device_age_hours,device_new_last_24h
0,tx_u1_2,u1,dev2,2025-09-24 10:00:00,500.0,2025-09-24 10:00:00,0.000000,True
5,tx_u1_6,u1,dev3,2025-09-24 10:02:00,500.0,2025-09-24 10:01:00,0.016667,True
17,tx_u2_2,u2,dev1,2025-09-24 10:05:00,500.0,2025-09-24 10:00:00,0.083333,True


In [ ]:

# Save the notebook programmatically (the nbformat part already assembled)
print("This notebook will be saved by the driver code that constructed it.")


This notebook will be saved by the driver code that constructed it.
